In [1]:
import uuid
import pandas
import numpy
from random import shuffle

import sklearn
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import sklearn.manifold

import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import os

In [2]:
def load_data():
    if os.path.isdir("/cluster/home/simondi/spark/"):
        filename = "/cluster/home/simondi/simondi/tix/data/screening_data/cells_sample_10_normalized.tsv"
    else:
        filename = "/Users/simondi/PHD/data/data/target_infect_x/screening_data_subset/cells_sample_10_normalized.tsv"

    data = pandas.read_csv(filename, sep="\t", header=0)

    feature_cols = [(i, x) for i, x in enumerate(data.columns.values) if
                    x.startswith("cells")]
    for i, c in feature_cols:
        data.loc[:, c] = data.loc[:, c].astype('float64')

    data_new = data.query(
      "library=='d' and design=='p' and replicate==1 and (pathogen=='brucella' or pathogen=='listeria' or pathogen=='adeno' or pathogen=='bartonella')").groupby(
      ["gene"]).filter(lambda x: len(x) == 40)

    del data_new["cells.children_invasomes_count"]
    del data_new["cells.children_bacteria_count"]

    X = data_new.dropna()
    return X

In [7]:
data = load_data()

In [8]:
data.groupby("pathogen").size()

pathogen
adeno         140850
bartonella    155620
brucella      155620
listeria      155620
dtype: int64

In [9]:
fl = "/Users/simondi/PROJECTS/target_infect_x_project/src/tix_util/tix_analysis/plots/scatter_gene_pathogen"

In [20]:
feats = [x for x in data.columns if x.startswith("cells")]
feats

['cells.areashape_area',
 'cells.areashape_eccentricity',
 'cells.areashape_extent',
 'cells.areashape_formfactor',
 'cells.areashape_majoraxislength',
 'cells.areashape_minoraxislength',
 'cells.areashape_perimeter',
 'cells.location_center_x',
 'cells.location_center_y',
 'cells.neighbors_anglebetweenneighbors_2',
 'cells.neighbors_firstclosestobjectnumber_2',
 'cells.neighbors_firstclosestxvector_2',
 'cells.neighbors_firstclosestyvector_2',
 'cells.neighbors_numberofneighbors_2',
 'cells.neighbors_secondclosestobjectnumber_2',
 'cells.neighbors_secondclosestxvector_2',
 'cells.neighbors_secondclosestyvector_2',
 'cells.parent_nuclei']

In [58]:
def plot_pairwise(X):
    markers = ['o', 'v','8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X' ]
    pathogens = list(set(X["pathogen"]))
    path_map = {}
    for i, p in enumerate(pathogens):
        path_map[p] = markers[i]
    
    uniq = list(set(X["gene"]))
    
    hot = plt.get_cmap('hot')
    cNorm = colors.Normalize(vmin=0, vmax=len(uniq))
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=hot)
    alpha = 0.5
    
    for c1 in feats:
        for c2 in feats:
            fig = plt.figure()
            for i in range(len(uniq)):
                indx = X["gene"] == uniq[i]
                XG = X.loc[indx,:]
                unip = list(set(XG["pathogen"]))
                for j in range(len(unip)):
                    indx = XG["pathogen"] == unip[j]
                    plt.scatter(XG.loc[indx, c1], XG.loc[indx, c2], 
                                color=scalarMap.to_rgba(i),
                                label=unip[j].capitalize() + ": "+ uniq[i].upper(),
                                marker=path_map[ unip[j]  ], alpha=alpha)
            if len(uniq) <= 5:     
                plt.legend( fontsize="xx-small")    
            plt.xlabel(c1)
            plt.ylabel(c2)
            plt.savefig(fl +  "_" +c1   + "_"  + c2 + "_" +uid+ ".png",      dpi=720)
            plt.close(fig)

In [60]:
plt.close()

In [61]:
cnt = 5
uid = str(uuid.uuid1())
used_genes = gene_list[:cnt]
X = data[ data["gene"].isin(used_genes)]
plot_pairwise(X)

/Users/simondi/anaconda3/envs/py35/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [10]:
def plot(X2, X, method):
    markers = [    'o', 'v','8',   's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X' ]
    pathogens = list(set(X["pathogen"]))
    path_map = {}
    for i, p in enumerate(pathogens):
        path_map[p] = markers[i]
    
    uniq = list(set(X["gene"]))
    
    hot = plt.get_cmap('hot')
    cNorm = colors.Normalize(vmin=0, vmax=len(uniq))
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=hot)
    alpha = 0.5
    
    for 
    
    plt.figure()

    
    
        indx = X["gene"] == uniq[i]
        XG = X.loc[indx,:]
        XP = X2[indx,:]
        unip = list(set(XG["pathogen"]))
        for j in range(len(unip)):
            indx = XG["pathogen"] == unip[j]
            plt.scatter(XP[indx, 1], XP[indx, 0], 
                        color=scalarMap.to_rgba(i),
                        label=unip[j].capitalize() + ": "+ uniq[i].upper(),
                        marker=path_map[ unip[j]  ], alpha=alpha)
    if len(uniq) <= 5:     
        plt.legend( fontsize="xx-small")    

    plt.savefig(fl +  "_" +  method + "_2d" +uid+ ".png",
      dpi=720)
    plt.close()

In [11]:
def plot_pca(Xtrans, X, method):
    markers = ['o', 'v','8',   's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X' ]
    pathogens = list(set(X["pathogen"]))
    path_map = {}
    for i, p in enumerate(pathogens):
        path_map[p] = markers[i]
    
    uniq = list(set(X["gene"]))
    
    hot = plt.get_cmap('hot')
    cNorm = colors.Normalize(vmin=0, vmax=len(uniq))
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=hot)
    alpha = 0.5
    
    X2 = Xtrans.transform(X.loc[:, feature_cols_idxs])
    
    fig = plt.figure()
    for i in range(len(uniq)):
        indx = X["gene"] == uniq[i]
        XG = X.loc[indx,:]
        XP = X2[indx,:]
        unip = list(set(XG["pathogen"]))
        for j in range(len(unip)):
            indx = XG["pathogen"] == unip[j]
            plt.scatter(XP[indx, 0], XP[indx, 1], 
                        color=scalarMap.to_rgba(i),
                        label=unip[j].capitalize() + ": "+ uniq[i].upper(),
                        marker=path_map[ unip[j]  ], alpha=alpha)
    plt.xlabel("Variance ratio:" + "{0:.2f}".format(Xtrans.explained_variance_ratio_[0]))
    plt.ylabel("Variance ratio:" + "{0:.2f}".format(Xtrans.explained_variance_ratio_[1]))
    if len(uniq) <= 5:     
        plt.legend( fontsize="xx-small")        
    plt.savefig(fl +  "_" +  method + "_2d" +uid+ ".png",
      dpi=720)
    plt.close(fig)

    fig, ax = plt.subplots()
    ax.bar(list(numpy.array(range(10)) + 0.5),  Xtrans.explained_variance_ratio_[:10], width=0.5)

    ax.set_ylabel('Variance ratio')
    ax.set_xticks(list(numpy.array(range(10)) + 0.5))
    ax.set_xticklabels(["PCA" + str(i+1) for i in range(10)])
    plt.savefig(fl + "_" + method + "_2d_scree"+uid+".png", dpi=720)
    plt.close(fig)

In [12]:
gene_list = list(set(data["gene"]))

In [13]:
feature_cols_idxs = [x for x in data.columns.values if x.startswith("cells")]

In [14]:
def tsne(data, genes_list, cnt):
    used_genes = gene_list[:cnt]
    X = data[ data["gene"].isin(used_genes)]
    for x in [1, 5, 10, 30, 50, 100]:
        ts2 = sklearn.manifold.TSNE(n_components=2, perplexity=x, n_iter=5000)
        X2 = ts2.fit_transform(X.loc[:, feature_cols_idxs])
        plot(X2, X, "tsne_" + str(x))

In [15]:
def lle(data, genes_list, cnt):
    used_genes = gene_list[:cnt]
    X = data[ data["gene"].isin(used_genes)]
    for x in [5, 10, 30]:
        ts2 = sklearn.manifold.LocallyLinearEmbedding(n_components=2,method="standard",n_neighbors=x)
        X2 = ts2.fit_transform(X.loc[:, feature_cols_idxs])
        plot(X2, X, "lle_"+ str(x))

In [16]:
def kpca(data, genes_list, cnt):
    used_genes = gene_list[:cnt]
    X = data[ data["gene"].isin(used_genes)]
    for x in ["rbf", "poly", "linear"]:
        ts2 = sklearn.decomposition.KernelPCA(n_components=2, kernel=x)
        X2 = ts2.fit_transform(X.loc[:, feature_cols_idxs])
        plot(X2, X, "kpca_" + x)

In [17]:
def pca(data, genes_list, cnt):
    used_genes = gene_list[:cnt]
    X = data[ data["gene"].isin(used_genes)]
    ts2 = sklearn.decomposition.PCA(n_components=11)
    X2 = ts2.fit(X.loc[:, feature_cols_idxs])
    plot_pca(X2, X, "pca")

In [18]:
for i in range(10):
    uid = str(uuid.uuid1())
    shuffle(gene_list)
    pca(data, gene_list, 5)
    tsne(data, gene_list, 5)
#    lle(data, gene_list, 5)
    kpca(data, gene_list, 5)
    
# pca(data, gene_list, 100)
# tsne(data, gene_list, 100)
# lle(data, gene_list, 100)
# kpca(data, gene_list, 100)

KeyboardInterrupt: 

In [ ]:
list(range(10))[:5]